<table>
    <h2>Conversion from Brazilian Standards (SNUC)</h2>
    <tr>
        <th>Proteção Integral (PI)</th>
        <td><a href="https://en.wikipedia.org/wiki/Ecological_station_(Brazil)">Estação ecológica</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Biological_reserve_(Brazil)">Reserva biológica</a></td>
        <td><a href="https://en.wikipedia.org/wiki/National_park">Parque nacional</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Natural_monument">Monumento natural</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Wildlife_refuge_(Brazil)">Refúgio de vida silvestre</a></td>
    </tr>
    <tr>
        <th>Uso Sustentável (US)</th>
        <td><a href="https://en.wikipedia.org/wiki/Area_of_relevant_ecological_interest_(Brazil)">Área de relevante interesse ecológico</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Private_natural_heritage_reserve_(Brazil)">Reserva particular do patrimônio natural</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Environmental_protection_area_(Brazil)">Área de proteção ambiental</a></td>
        <td><a href="https://en.wikipedia.org/wiki/National_forest_(Brazil)">Floresta nacional</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Sustainable_development_reserve_(Brazil)">Reserva de desenvolvimento sustentável</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Wildlife_reserve_(Brazil)">Reserva de fauna</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Extractive_reserve">Reserva extrativista</a></td>
    </tr>
</table>

<h2>Source of Data</h2>

Populational Data - Source: <a href="https://sidra.ibge.gov.br/tabela/6579">IBGE</a>

Protected Areas - Source: i3Geo - Ministry of the Environment (<a href="http://mapas.mma.gov.br/i3geo/datadownload.htm">i3Geo > Áreas Especiais > Unidades de conservação > Unidades de conservação (todas)</a>)<br>(Direct download link: <a href="http://mapas.mma.gov.br/ms_tmp/ucstodas.shp"> .shp</a>, <a href="http://mapas.mma.gov.br/ms_tmp/ucstodas.shx">.shx</a> and <a href="http://mapas.mma.gov.br/ms_tmp/ucstodas.dbf">.dbf</a>)

Paramters for script execution

>About the requests

>>Tip: If you want to search more than one type of protected area (i.e: PI and US), write as following:
<b>grupo_UC = ['PI','US']</b>
<a href="https://www.mma.gov.br/areas-protegidas/unidades-de-conservacao/categorias.html">Categories of PA by SNUC (Brazil-Standard)</a>

In [37]:
#Type of protected area
grupo_UC = ['PI']

>>Tip: in this paramters, the first value is the minimum year of creation, and the second, the maximum. If you want to search all protected areas created in 2008, write as following:
<b>ano_criação = ('2008','2008')</b>

In [38]:
#creating year of protected area
ano_criação = ('2005','2015') 

>About database

In [39]:
#Table that storage the results of search in the database
tabela_resultados = 'perfil_uc_2005_2015_2'

In [40]:
#user/password
conexao_banco = ('administrador-banco','senha')

>About the output data

In [41]:
#name of file that contains all data processed (it will be always a .csv file)
arquivo_exportado = 'results.csv'

The algorithm

In [42]:
#dependencies
import psycopg2
import time
import statistics
import os
import pandas

In [43]:
#routine to fetch the query and create a list with all the results
def fetch_query(query):
	cur.execute(query)
	try:
		ret = cur.fetchall()
		return [p[0] for p in ret]
	except:
		print("fail!")
		return []

Database stuff

In [44]:
#create the database(DB) connection
con = psycopg2.connect(host='localhost', database='postgres',
user=conexao_banco[0], password=conexao_banco[1])
cur = con.cursor()

#queries
grupos_procurados = '('+'OR'.join([" grupo4 = '"+g+"'" for g in grupo_UC])+')'
unidades_conservacao = "select nome_uc1 from \"br.domicilios.mma\".ucstodas_final where "+grupos_procurados+" and ano_cria6 >= '"+ano_criação[0]+"' and ano_cria6 <= '"+ano_criação[1]+"'"
cidades_grandes = "select nome_uc1 from \"br.domicilios.mma\".ucstodas_final where "+grupos_procurados+" and ano_cria6 >= '"+ano_criação[0]+"' and ano_cria6 <= '"+ano_criação[1]+"'"

Queries stuff

In [45]:
#Get all Primary Keys (PK) from the Protected Areas (PA) (The PKs on this table is the name of each PA)
recset = fetch_query(unidades_conservacao)
#list of queries to be done
queries = []
#list of execution time of each query (we use this for the statistics)
tempo = []
#progress of the queries (list of queries that was already done)
ja_feito = []

In [46]:
#Load all queries that already executed (if is the first time, this file is empty,so it will read nothing)
try:
    with open("uc_feitas.txt","r",encoding='utf-8') as f:
        for line in f:
            print(line[:-1],"already processed")
            ja_feito.append(line[:-1])
except FileNotFoundError:
    #create the file and leave it in blank
    open('uc_feitas.txt','w+',encoding='utf-8').close()

Execute the search query for each PA's ID that was selected before

In [47]:
try:
    cur.execute("SELECT * from \"br.domicilios.mma\"."+tabela_resultados)
    con.commit()
except psycopg2.errors.UndefinedTable:
    con.commit()
    print('Creating the table...')
    cur.execute("CREATE TABLE \"br.domicilios.mma\"."+tabela_resultados+"(\
            gid INTEGER,\
           geocodigo VARCHAR (17),\
           uf VARCHAR (19),\
           municipio VARCHAR (33),\
           habitantes numeric,\
           nome_uc VARCHAR (255),\
           distancia_ VARCHAR (21),\
           geom GEOMETRY\
               )")
    con.commit()


In [ ]:
#for each PA
for (i,rec) in enumerate(recset):
	print("Fetching %s..."%rec)
    #skip the already processed PAs
	if(rec in ja_feito):
		print(rec,'already done. Skipping...')
		continue
	start = time.time()
    
	#Execute the query as our paramenters that was set before
	cur.execute("INSERT INTO \"br.domicilios.mma\"."+tabela_resultados+"\
		SELECT a.gid, a.geocodigo, \
       CASE\
        WHEN (a.estado = '11' )  THEN 'rondônia'\
        WHEN (a.estado = '12' )  THEN 'acre'\
        WHEN (a.estado = '13' )  THEN 'amazonas'\
        WHEN (a.estado = '14' )  THEN 'roraima'\
        WHEN (a.estado = '15' )  THEN 'pará'\
        WHEN (a.estado = '16' )  THEN 'amapá'\
	        WHEN (a.estado = '17' )  THEN 'tocantins'\
	        WHEN (a.estado = '21' )  THEN 'maranhão'\
	        WHEN (a.estado = '22' )  THEN 'piauí'\
	        WHEN (a.estado = '23' )  THEN 'ceará'\
	        WHEN (a.estado = '24' )  THEN 'rio grande do norte'\
	        WHEN (a.estado = '25' )  THEN 'paraíba'\
	        WHEN (a.estado = '26' )  THEN 'pernanbuco'\
	        WHEN (a.estado = '27' )  THEN 'alagoas'\
	        WHEN (a.estado = '28' )  THEN 'sergipe'\
	        WHEN (a.estado = '29' )  THEN 'bahia'\
	        WHEN (a.estado = '31' )  THEN 'minas gerais'\
	        WHEN (a.estado = '32' )  THEN 'espírito santo'\
	        WHEN (a.estado = '33' )  THEN 'rio de janeiro'\
	        WHEN (a.estado = '35' )  THEN 'são paulo'\
	        WHEN (a.estado = '41' )  THEN 'paraná'\
	        WHEN (a.estado = '42' )  THEN 'santa catarina'\
	        WHEN (a.estado = '43' )  THEN 'rio grande do sul'\
	        WHEN (a.estado = '50' )  THEN 'mato grosso do sul'\
	        WHEN (a.estado = '51' )  THEN 'mato grosso'\
	        WHEN (a.estado = '52' )  THEN 'goiás'\
	        WHEN (a.estado = '53' )  THEN 'distrito federal'\
	        WHEN (a.estado = '00' )  THEN 'chico mendes'\
	    END AS UF\
	, a.nome as municipio, a.habitantes, '"+rec+"',\
	       to_char(ST_DistanceSphere(\
	                    a.geom, (SELECT b.geom FROM \"br.domicilios.mma\".ucstodas_final b where b.nome_uc1 = \'"+rec+"\')\
	                  ) /1000,'99999999999999999D99') AS \"distancia_km\", a.geom\
	  FROM \"br.domicilios.mma\".municipios_perfil_parsec a\
		WHERE \
		    (ST_DistanceSphere(a.geom, (SELECT b.geom FROM \"br.domicilios.mma\".ucstodas_final b where b.nome_uc1 = \'"+rec+"\')) /1000) <= 100  \
		ORDER BY\
		  a.geom <#> (SELECT c.geom FROM \"br.domicilios.mma\".ucstodas_final c where c.nome_uc1 = \'"+rec+"\');")
	#apply the modifications
	con.commit()    
	'''
	The file below stores the PKs of all the processed queries, 
	in order to avoid duplicate cases if this script needs
	to restart all the search.
	'''
	f = open("uc_feitas.txt","a+",encoding='utf-8')
	f.write(rec+"\n")
	f.close()
	fim = time.time()
    
	'''
	Note: Since all this process have a great cost
	in terms of computer instructions, this could 
	request a considerable amount of time in 
	average computers. 

	Don't be scarred if the remaning time value reaches a high value :)
	(the time of each query execution is about 2 minutes at Core i7-4790 3.60Ghz)
	'''
	#calculate and report the statistics about the program
	tempo.append((fim - start))
	v = int(statistics.mean(tempo))*(len(recset)-i)
	print("%d/%d Done. About "%(i+1,len(recset)),int(v/3600)," Hours",int(v%3600)/60,"Minutes left")
    

Fetching REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUNÃ E DO PADRE...
1/237 Done. About  3  Hours 17.5 Minutes left
Fetching PARQUE NATURAL MUNICIPAL DO CANCãO...
2/237 Done. About  2  Hours 1.9333333333333333 Minutes left
Fetching PARQUE ESTADUAL GUARIBA...
3/237 Done. About  1  Hours 41.833333333333336 Minutes left
Fetching PARQUE ESTADUAL SUCUNDURI...
4/237 Done. About  1  Hours 45.3 Minutes left
Fetching PARQUE ESTADUAL DO MATUPIRI...
5/237 Done. About  1  Hours 40.96666666666667 Minutes left
Fetching PARQUE ESTADUAL PONTA DA TULHA...
6/237 Done. About  1  Hours 32.8 Minutes left
Fetching MUNUMENTO NATURAL DOS CANIONS DO SUBAÉ...
7/237 Done. About  1  Hours 32.4 Minutes left
Fetching ESTAÇÃO ECOLÓGICA DO RIO PRETO...
8/237 Done. About  1  Hours 28.166666666666668 Minutes left
Fetching PARQUE ESTADUAL DA SERRA DOS MONTES ALTOS...
9/237 Done. About  1  Hours 54.5 Minutes left
Fetching REFUGIO DE VIDA SILVESTRE DA SERRA DOS MONTES ALTOS...
10/237 Done. About  2  Hours 32.0 Minutes le

Prepare and export the reults into a csv file

>Export the raw table into csv data

In [ ]:
#get the current directory
curr_directory = os.getcwd()

#ensure the csv format
if(arquivo_exportado[-4:] != '.csv'):
    arquivo_exportado += '.csv'

#path to the file
file_name = curr_directory+"/"+arquivo_exportado

#file that contains some cache from our operations
temp_file_name = file_name+'.temp'

#list of columns to save
columns_save = ['gid', 'geocodigo', 'uf', 'municipio', 'habitantes', 'nome_uc', 'distancia_']

#export all data to temporary file
with open(temp_file_name,'w+',encoding='utf-8') as f:
    f.write(';'.join(columns_save)+'\n')
    cur.copy_to(f, '"br.domicilios.mma".'+tabela_resultados, columns=tuple(columns_save),sep=';')
    con.commit()

>Change the table columns into: 
<br><b>GID</b>: the ID
<br><b>Geocodigo</b>: Geocode based on IBGE rules
<br><b>UF</b>: Unidades Federativas (Federative Unites, the Subdivisions of Brazil)
<br><b>Municipio</b>: Municipality
<br><b>Habitantes</b>: size of Population
<br><b>UCs_num</b>: Number of protected areas around the Municipality
<br><b>UCs_nome</b>: Name of each protected areas.
<br><b>Distancias_km</b>: distances (in km), separated by ',' (comma), for each Protected Area, in the same order of the column <b>UCs_nome</b>
<br>
<br>Is important to note, the value of column <b>UCs_num</b> is the size of the column <b>UCs_nome</b> and <b>Distancias_</b>

In [ ]:
read_content = pandas.read_csv(temp_file_name, delimiter = ';', encoding='utf-8')

#store GIDs, Distances, and PA's Names
distances_GIDs = {}
names_GIDs = {}
for gid,distance,name in zip(read_content['gid'],read_content['distancia_'],read_content['nome_uc']):
    if(gid in distances_GIDs):
        distances_GIDs[gid].append(str(distance))
    else:
        distances_GIDs[gid] = [str(distance)]
    if(gid in names_GIDs):
        names_GIDs[gid].append(name)
    else:
        names_GIDs[gid] = [name]

processed_GIDs = []
with open(file_name,'w+',encoding='utf-8') as f:
    #header of csv
    f.write('gid;geocodigo;uf;habitantes;municipio;UCs_num;distancias_km\n')
    for content in zip(read_content['gid'],read_content['geocodigo'],read_content['uf'],read_content['municipio'],read_content['habitantes']):
        if(not content[0] in processed_GIDs):
            processed_GIDs.append(content[0])
            c = [str(s) for s in content]
            column_UCs_num = str(len(distances_GIDs[content[0]]))
            column_UCs_nome = ','.join(names_GIDs[content[0]])
            column_Distancias_km = ','.join(distances_GIDs[content[0]])
            buffer = ';'.join(c)+';'+column_UCs_num+';'+column_UCs_nome+'\n'
            f.write(buffer)

print('Export the output data into',file_name)
#pandas.read_csv(file_name, delimiter = ';', encoding='utf-8')

<h2>Example of Operation</h2>
<br>Selecting the Municipalities from <a href="https://pt.wikipedia.org/wiki/Vale_do_Ribeira">Vale do Ribeira</a>

In [ ]:
Vale_do_Ribeira = {
    'Alto Ribeira'  :['Barra do Chapéu','Itapirapuã Paulista','Apiaí','Itaóca','Iporanga','Ribeira','Adrianópolis'],
    'Baixo Ribeira' :['Apiaí','Iporanga','Eldorado','Sete Barras'],
    'Rio Ribeira de Iguape' :['Registro','Pariquera-Açu','Iguape'],
    'Alto Juquiá' : ['São Lourenço da Serra','Juquitiba','Tapiraí'],
    'Médio Juquiá' : ['Tapiraí','Juquiá','Miracatu'],
    'Baixo Juquiá' : ['Juquiá','Tapiraí','Sete Barras'],
    'Rio São Lourenço' : ['Miracatu','Pedro de Toledo','Juquiá'],
    'Rio Itariri': ['Itariri','Pedro de Toledo'],
    'Rio Una da Aldeia' : ['Iguape'],
    'Rio Pardo' : ['Barra do Turvo'],
    'Rio Jacupiranga' : ['Jacupiranga','Pariquera-Açu','Cajati','Registro'],
    'Vertente Marítima Sul' : ['Cananéia','Ilha Comprida'],
    'Vertente Marítima Norte' : ['Iguape']
}


municipios = []
for região in Vale_do_Ribeira:
    municipios += Vale_do_Ribeira[região]

#ensure they are all lowercase
municipios = [name.lower() for name in municipios]

#query
print('SELECT * FROM "br.domicilios.mma".perfil_uc_2005_2015_2 where municipio = '+' OR municipio = '.join(["'"+m+"'" for m in municipios]))

read_content = pandas.read_csv(file_name, delimiter = ';', encoding='utf-8')

file_name_ribeira = curr_directory+'/vale_do_ribeira.csv'
with open(file_name_ribeira,'w+',encoding='utf-8') as f:
    #header of csv
    f.write('gid;geocodigo;uf;habitantes;municipio;UCs_num;distancias_km\n')
    for content in zip(read_content['gid'],read_content['geocodigo'],read_content['uf'],read_content['habitantes'],read_content['municipio'],read_content['UCs_num'],read_content['UCs_nome'],read_content['distancias_km']):
        if(content[4] in municipios):
            c = [str(s) for s in content]
            buffer = ';'.join(c)+'\n'
            f.write(buffer)

print('Export the output data into',file_name_ribeira)
pandas.read_csv(file_name_ribeira, delimiter = ';', encoding='utf-8')   

End of the queries, close the program

In [ ]:
#Close the DB connection
con.close()
print("End :)")
exit(1)